In [1]:
# rm(list=ls())
require(data.table)
require(MASS)
require(ggplot2)
# require(foreach)
require(gridExtra)
require(stringr)
require(matrixStats)
require(parallel)
require(abind)

parallel::detectCores()
## Resizing notebook plot space
options(repr.plot.width=16, repr.plot.height=9)

Loading required package: data.table
Loading required package: MASS
Loading required package: ggplot2
Loading required package: gridExtra
Loading required package: stringr
Loading required package: matrixStats
Loading required package: parallel
Loading required package: abind


[1] 64

In [2]:
## POP DATA from UN
# pop <- data.table(fread("/home/j/Project/IRH/Forecasting/data/WPP_forecasted_population.csv"))
pop <- fread('/ihme/forecasting/data/pop/20150101_wpp/data.csv')
pop <- pop[, age_group_id:=15+5*(age_group_id -8)] 
pop<- pop[age_group_id == 15 | age_group_id == 20 | age_group_id == 25 | age_group_id == 30 | age_group_id == 35 | age_group_id == 40 | age_group_id == 45 | age_group_id == 50 | age_group_id == 55 | age_group_id == 60 | age_group_id == 65 | age_group_id == 70 | age_group_id == 75 | age_group_id == 85,]
head(pop)

location_id,sex_id,age_group_id,year_id,pop
6,1,15,1950,27530292
6,1,15,1951,28008975
6,1,15,1952,28183766
6,1,15,1953,28252474
6,1,15,1954,28303795
6,1,15,1955,28343730


In [3]:
## Education forecasts
# edu_orig<-data.table(fread(paste0("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/tabulated_data/archive_nafis/20161121_GBD2016prelim_95+_raked_2/gpr_draws.csv"), header = T))
# edu_orig <- edu_orig[, V1:=NULL]
# head(edu_orig)

edu_ref <- fread("/ihme/forecasting/data/fbd_scenarios_data/forecast/covariate/education/20170608_GBD2016Final/20170608_GBD2016Final_gpr_draws_scenario0.csv")
edu_pes <- fread("/ihme/forecasting/data/fbd_scenarios_data/forecast/covariate/education/20170608_GBD2016Final/20170608_GBD2016Final_gpr_draws_scenario-1.csv")
edu_opt <- fread("/ihme/forecasting/data/fbd_scenarios_data/forecast/covariate/education/20170608_GBD2016Final/20170608_GBD2016Final_gpr_draws_scenario1.csv")

Read 475800 rows and 1004 (of 1004) columns from 6.191 GB file in 00:02:55
Read 475800 rows and 1004 (of 1004) columns from 6.196 GB file in 00:02:52
Read 475800 rows and 1004 (of 1004) columns from 6.190 GB file in 00:02:59


In [4]:
## Convert age_group_id to age_start
  ## Leave out the younglings (no maternal education)    
    edu_ref <- edu_ref[age_group_id >7]
    edu_ref <- edu_ref[, age_group_id:=15+5*(age_group_id -8)] 
    
    edu_pes <- edu_pes[age_group_id >7]
    edu_pes <- edu_pes[, age_group_id:=15+5*(age_group_id -8)] 
    
    edu_opt <- edu_opt[age_group_id >7]
    edu_opt <- edu_opt[, age_group_id:=15+5*(age_group_id -8)] 



# edu_orig_2 <- edu_orig
## Merge pop for later stuff
# edu_orig_2 <- merge(edu_orig_2, pop, by = c("location_id", "year_id", "age_group_id", "sex_id"))
# head(edu_orig_2)

In [5]:
## How you lookin?
 head(edu_ref); head(edu_opt); head(edu_pes);

location_id,age_group_id,sex_id,year_id,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,⋯,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
6,15,1,1980,8.195544,7.856324,8.401491,9.273246,8.992329,8.964878,⋯,9.161543,8.760984,8.799634,7.828814,8.363295,8.391927,7.986652,8.969120,7.888389,8.868681
6,15,1,1981,8.147565,7.995990,8.300895,9.233780,9.023098,8.942048,⋯,9.096156,8.771453,8.835323,7.893521,8.456012,8.404336,8.119655,8.888441,7.918112,8.809835
6,15,1,1982,8.099316,8.128367,8.207957,9.189589,9.047270,8.918667,⋯,9.027742,8.785263,8.869407,7.953739,8.548425,8.412889,8.246784,8.807993,7.947025,8.759813
6,15,1,1983,8.052140,8.253047,8.123502,9.141470,9.065569,8.895725,⋯,8.957377,8.802851,8.903085,8.010255,8.640162,8.418698,8.368072,8.729619,7.975628,8.719149
6,15,1,1984,8.007288,8.369833,8.048323,9.090263,9.078829,8.874083,⋯,8.886159,8.824562,8.937524,8.063889,8.730709,8.422927,8.483543,8.655100,8.004449,8.688123
6,15,1,1985,7.965937,8.478749,7.983198,9.036858,9.087964,8.854468,⋯,8.815209,8.850664,8.973828,8.115483,8.819436,8.426788,8.593215,8.586134,8.034035,8.666778


location_id,age_group_id,sex_id,year_id,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,⋯,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
6,15,1,1980,8.195544,7.856324,8.401491,9.273246,8.992329,8.964878,⋯,9.161543,8.760984,8.799634,7.828814,8.363295,8.391927,7.986652,8.969120,7.888389,8.868681
6,15,1,1981,8.147565,7.995990,8.300895,9.233780,9.023098,8.942048,⋯,9.096156,8.771453,8.835323,7.893521,8.456012,8.404336,8.119655,8.888441,7.918112,8.809835
6,15,1,1982,8.099316,8.128367,8.207957,9.189589,9.047270,8.918667,⋯,9.027742,8.785263,8.869407,7.953739,8.548425,8.412889,8.246784,8.807993,7.947025,8.759813
6,15,1,1983,8.052140,8.253047,8.123502,9.141470,9.065569,8.895725,⋯,8.957377,8.802851,8.903085,8.010255,8.640162,8.418698,8.368072,8.729619,7.975628,8.719149
6,15,1,1984,8.007288,8.369833,8.048323,9.090263,9.078829,8.874083,⋯,8.886159,8.824562,8.937524,8.063889,8.730709,8.422927,8.483543,8.655100,8.004449,8.688123
6,15,1,1985,7.965937,8.478749,7.983198,9.036858,9.087964,8.854468,⋯,8.815209,8.850664,8.973828,8.115483,8.819436,8.426788,8.593215,8.586134,8.034035,8.666778


location_id,age_group_id,sex_id,year_id,draw_0,draw_1,draw_2,draw_3,draw_4,draw_5,⋯,draw_990,draw_991,draw_992,draw_993,draw_994,draw_995,draw_996,draw_997,draw_998,draw_999
6,15,1,1980,8.195544,7.856324,8.401491,9.273246,8.992329,8.964878,⋯,9.161543,8.760984,8.799634,7.828814,8.363295,8.391927,7.986652,8.969120,7.888389,8.868681
6,15,1,1981,8.147565,7.995990,8.300895,9.233780,9.023098,8.942048,⋯,9.096156,8.771453,8.835323,7.893521,8.456012,8.404336,8.119655,8.888441,7.918112,8.809835
6,15,1,1982,8.099316,8.128367,8.207957,9.189589,9.047270,8.918667,⋯,9.027742,8.785263,8.869407,7.953739,8.548425,8.412889,8.246784,8.807993,7.947025,8.759813
6,15,1,1983,8.052140,8.253047,8.123502,9.141470,9.065569,8.895725,⋯,8.957377,8.802851,8.903085,8.010255,8.640162,8.418698,8.368072,8.729619,7.975628,8.719149
6,15,1,1984,8.007288,8.369833,8.048323,9.090263,9.078829,8.874083,⋯,8.886159,8.824562,8.937524,8.063889,8.730709,8.422927,8.483543,8.655100,8.004449,8.688123
6,15,1,1985,7.965937,8.478749,7.983198,9.036858,9.087964,8.854468,⋯,8.815209,8.850664,8.973828,8.115483,8.819436,8.426788,8.593215,8.586134,8.034035,8.666778


In [6]:
### Let's concatenatate the age and sex columns to be one identifier:
data_cleanup_1 <- function(dt) {
    dt <- dt[, age_sex := paste0(age_group_id, "_", sex_id)]
    dt <- dt[, age_group_id:= NULL]
    dt <- dt[, sex_id:= NULL]
    dt <- setcolorder(dt, c("location_id", "year_id", "age_sex", paste0("draw_",c(0:999)) ))
    head(dt)
    
    return(dt)
}

edu_ref <- data_cleanup_1(edu_ref)
edu_pes <- data_cleanup_1(edu_pes)
edu_opt <- data_cleanup_1(edu_opt)

### Reshape long by draws, wide by year, use copula with spearman = 1, and reshape back AND collapse by country-year

In [13]:
# Make draws long first
system.time(edu_ref_long <- melt(edu_ref, id.vars = c("location_id", "age_sex", "year_id"), value.name = "edu", varnames = "draw"))
system.time(edu_pes_long <- melt(edu_pes, id.vars = c("location_id", "age_sex", "year_id"), value.name = "edu", varnames = "draw"))
system.time(edu_opt_long <- melt(edu_opt, id.vars = c("location_id", "age_sex", "year_id"), value.name = "edu", varnames = "draw"))

   user  system elapsed 
  6.511   1.399   7.912 

   user  system elapsed 
  8.822 240.254 257.161 

   user  system elapsed 
  5.043 234.146 245.492 

In [23]:
## Number of countries we have to array over
countries <- unique(edu_ref[year_id ==1980, location_id])
countries
length(countries)

[1]   6   7   8  10  11  12  13  14  15  16  17  18  19  20  22  23  24  25
 [19]  26  27  28  29  30  33  34  35  36  37  38  39  40  41  43  44  45  46
 [37]  47  48  49  50  51  52  53  54  55  57  58  59  60  61  62  63  66  67
 [55]  68  69  71  72  74  75  76  77  78  79  80  81  82  83  84  85  86  87
 [73]  88  89  90  91  92  93  94  95  97  98  99 101 102 105 106 107 108 109
 [91] 110 111 112 113 114 115 116 117 118 119 121 122 123 125 126 127 128 129
[109] 130 131 132 133 135 136 139 140 141 142 143 144 145 146 147 148 149 150
[127] 151 152 153 154 155 156 157 160 161 162 163 164 165 168 169 170 171 172
[145] 173 175 176 177 178 179 180 181 182 183 184 185 186 187 189 190 191 193
[163] 194 195 196 197 198 200 201 202 203 204 205 206 207 208 209 210 211 212
[181] 213 214 215 216 217 218 298 305 349 351 376 385 422 435 522

[1] 195

In [17]:
## Ease up on the memory
rm(edu_ref); rm(edu_pes); rm(edu_opt)

In [18]:
## Get our baseline correlation matrix
load("/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/tabulated_data/archive_nafis/insample_corr_mat.Rdata")

## For now, trim the corr matrix to have 14 age groups
corr_mat <- corr_mat[1:28,1:28]

In [ ]:
## Convert them all to multidim array in the dims: [loc, year, age_sex, draw_num, data]

## Reference
system.time(edu_ref_array<- mclapply(countries, 
                                function(x) {reshape2::acast(edu_ref_long[location_id == x,], 
                                                      location_id ~  year_id ~ age_sex ~ variable, 
                                                      value.var = "edu") }, 
                                                        mc.cores = 4))
edu_ref_array <- abind(edu_ref_array, along=1)
str(edu_ref_array)

In [39]:
tail(edu_pes_long)

location_id,age_sex,year_id,variable,edu
522,80_2,2035,draw_999,-0.1569604
522,80_2,2036,draw_999,-0.1532036
522,80_2,2037,draw_999,-0.1466647
522,80_2,2038,draw_999,-0.1372289
522,80_2,2039,draw_999,-0.1248067
522,80_2,2040,draw_999,-0.1093387


In [ ]:
## Pessimistic
system.time(edu_pes_array<- mclapply(countries, 
                                function(x) {reshape2::acast(edu_pes_long[location_id == x,], 
                                                      location_id ~  year_id ~ age_sex ~ variable, 
                                                      value.var = "edu") }, 
                                                        mc.cores = 10))
edu_pes_array <- abind(edu_pes_array, along=1)
str(edu_pes_array)

In [ ]:
## Optimistic
system.time(edu_opt_array<- mclapply(countries, 
                                function(x) {reshape2::acast(edu_opt_long[location_id == x,], 
                                                      location_id ~  year_id ~ age_sex ~ variable, 
                                                      value.var = "edu") }, 
                                                        mc.cores = 10))
edu_opt_array <- abind(edu_opt_array, along=1)
str(edu_opt_array)

### Neal's (TM) Multivariate Copula Function

In [33]:
draw2Dcopula <- function(X, cor_mat, df_return = F){
  L <- dim(X)[2]
  D <- dim(X)[3]
  Xsum <- apply(X, c(2, 3), sum)
  mvdat <- mvrnorm(n=D, mu=0 * 1:L, Sigma=cor_mat, empirical=TRUE)
  ranks <- apply(mvdat, 2, rank, ties.method="first")
  sortedXsim <- apply(Xsum, 1, function(x) sort(x, index.return=TRUE)$ix)
  sortedX <- X
  for(i in 1:L){
    sortedX[,i,] <- X[,i,sortedXsim[,i]]
  }
  Xcorr <- sortedX
  for(i in 1:L){
    Xcorr[,i,] <- sortedX[,i,ranks[,i]]
  }
  if (df_return==T) {
    return(data.table(melt(Xcorr)))
    }
      else {
          Xcorr
      }
}

### Let's 2D copulate the data now

In [34]:
system.time(edu_ref_corr_array <- mclapply(countries, 
                                    function(x) {cbind(x, draw2Dcopula(edu_ref_array[paste0(x),,,],
                                                                      corr_mat , df_return = T))}, mc.cores = 10 ))
                                        
edu_ref_corr_df <- do.call(rbind, edu_ref_corr_array)
                                        
colnames(edu_ref_corr_df) <- c("location_id", "year_id", "age_sex", "draw_num", "edu")
head(edu_ref_corr_df)    


system.time(edu_pes_corr_array <- mclapply(countries, 
                                    function(x) {cbind(x, draw2Dcopula(edu_pes_array[paste0(x),,,],
                                                                      corr_mat , df_return = T))}, mc.cores = 10 ))
                                        
edu_pes_corr_df <- do.call(rbind, edu_pes_corr_array)
                                        
colnames(edu_pes_corr_df) <- c("location_id", "year_id", "age_sex", "draw_num", "edu")
head(edu_pes_corr_df)    



system.time(edu_opt_corr_array <- mclapply(countries, 
                                    function(x) {cbind(x, draw2Dcopula(edu_opt_array[paste0(x),,,],
                                                                      corr_mat , df_return = T))}, mc.cores = 10 ))
                                        
edu_opt_corr_df <- do.call(rbind, edu_opt_corr_array)
                                        
colnames(edu_opt_corr_df) <- c("location_id", "year_id", "age_sex", "draw_num", "edu")
head(edu_opt_corr_df)

Warning message in mclapply(countries, function(x) {:
“all scheduled cores encountered errors in user code”

   user  system elapsed 
145.080  63.082   8.384 

ERROR: Error in dimnames(x) <- dn: length of 'dimnames' [2] not equal to array extent


### Collapse to year and country only

In [ ]:
# Reshape wide
Xcorr_df_with_pop <- dcast(Xcorr_df, location_id + year_id + age_sex ~ draw_num, value.var = c("edu"))
head(Xcorr_df_with_pop)

In [ ]:
# Split back into age and sex
Xcorr_df_with_pop2 <- Xcorr_df_with_pop
system.time({ Xcorr_df_with_pop2[, c("age_group_id","sex_id") := data.table(str_split_fixed(age_sex,"_",2))] })
Xcorr_df_with_pop2[, age_sex:=NULL]
head(Xcorr_df_with_pop2)
str(Xcorr_df_with_pop2)

In [ ]:
# Merge in population
Xcorr_df_with_pop2 <- data.table(merge(x=Xcorr_df_with_pop2, y=pop, by = c("location_id", "year_id", "age_group_id", "sex_id")))
head(Xcorr_df_with_pop2)

In [ ]:
# Collapse correlated data and compute the mean and PCs
Xcorr_df_CY<- Xcorr_df_with_pop2[, lapply(.SD, function(x) sum(x*pop)/sum(pop)), 
    by=c("location_id", "year_id"), .SDcols=cbind(paste0("draw_",c(0:999))) ] 
Xcorr_df_CY <- Xcorr_df_CY[, `:=` (mean = rowMeans(.SD, na.rm=T),
                                  lower = rowQuantiles(.SD, na.rm=T, probs=c(0.025)),
                                  upper = rowQuantiles(.SD, na.rm=T, probs=c(0.975))), 
                           .SDcols = c(paste0("draw_",c(0:999))) ]    
head(Xcorr_df_CY)

In [ ]:
# Collapse un-correlated data
uncorr_df_CY<- edu_orig_2[,lapply(.SD, function(x) sum(x*pop)/sum(pop)), 
    by=c("location_id", "year_id"), .SDcols=cbind(paste0("draw_",c(0:999))) ] 
uncorr_df_CY <- uncorr_df_CY[, `:=` (mean = rowMeans(.SD, na.rm=T),
                              lower = rowQuantiles(.SD, na.rm=T, probs=c(0.025)),
                              upper = rowQuantiles(.SD, na.rm=T, probs=c(0.975))), 
                       .SDcols = c(paste0("draw_a",c(0:999))) ] 
head(uncorr_df_CY)

#### Test plots

In [ ]:
# Convert to arrays for super faster subsetting
system.time(myarray3<- lapply(countries, 
                                function(x) reshape2::acast(Xcorr_df[location_id == x,], 
                                                      location_id ~  year_id ~ age_sex ~ draw_num, 
                                                      value.var = "edu")))
myarray3 <- abind(myarray3, along=1)
str(myarray3)

In [ ]:
x1<-data.frame(myarray2["6","2040","15_1",]); colnames(x1) <- "data"
x2<-data.frame(myarray3["6","2040","15_1",]); colnames(x2) <- "data"

ggplot(data = x1, aes(x=data)) + 
    geom_histogram(aes(y=  ..density..), alpha=0.3, fill="steelblue") +
    geom_density(color ="blue4") +
geom_histogram(data = x1, aes(x=data, y=  ..density..), alpha=0.15, fill="red1") +
    geom_density(color ="red4")   + ggtitle("China: age group 15-20; male")

In [ ]:
x1<-data.frame(myarray2["6","2040","75_2",]); colnames(x1) <- "data"
x2<-data.frame(myarray3["6","2040","75_2",]); colnames(x2) <- "data"

ggplot(data = x1, aes(x=data)) + 
    geom_histogram(aes(y=  ..density..), alpha=0.3, fill="steelblue") +
    geom_density(color ="blue4") +
geom_histogram(data = x1, aes(x=data, y=  ..density..), alpha=0.15, fill="red1") +
    geom_density(color ="red4") + ggtitle("China: age group 75-80; female")

In [ ]:
## Turn the collapsed ones into arrays as well
array_Xcorr_df_CY <- melt(Xcorr_df_CY, id.vars = c("location_id", "year_id"), value.name = "edu", variable.name = "variable")
array_Xcorr_df_CY <- reshape2::acast(array_Xcorr_df_CY, location_id ~ year_id ~ variable, value.var = "edu")
str(array_Xcorr_df_CY) ### last 3 values are mean, upper, lower

array_uncorr_df_CY <- melt(uncorr_df_CY, id.vars = c("location_id", "year_id"), value.name = "edu", variable.name = "variable")
array_uncorr_df_CY <- reshape2::acast(array_uncorr_df_CY, location_id ~ year_id ~ variable, value.var = "edu")
str(array_uncorr_df_CY) ### last 3 values are mean, upper, lower

In [ ]:
plot_country_collapsed<-function(loc_id) {
ggplot(uncorr_df_CY[location_id == loc_id,], aes(x = year_id, y = mean, ymin = lower, ymax = upper)) +
    geom_ribbon(alpha = 0.3, fill="steelblue") +
    geom_line(color = "blue4") +
    geom_ribbon(data=Xcorr_df_CY[location_id == loc_id,], aes(x = year_id, ymin = lower, ymax = upper),
                alpha = 0.15, fill="red1") +
    geom_line(aes(x=year_id, y=mean),color = "red4") + xlab("Year") + ylab("Education") +
    ggtitle(paste0("location_id: ", loc_id, "; collapsed to C-Y : blue before copula, red after copula"))
}

plot_country_collapsed(6)

In [ ]:
## Save out data

# Collapsed C-Y
fwrite(Xcorr_df_CY, "/home/j/WORK/01_covariates/02_inputs/education/update_2017/data/output_data/20161121_GBD2016prelim_95+_raked_2/country_year_A_S_T_collapsed.csv")

### Make country-year plots

In [ ]:
filename <- paste0("/homes/sadatnfs/test_edu.pdf")
pdf(filename, width=10, height = 6)
    for(c in countries){
        print(plot_country_collapsed(c))
    }
dev.off()